In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime

# Load the data
df = pd.read_csv('internship_data.csv')

def extract_numeric_duration(duration_str):
    """
    Extract numeric value from duration strings like "4 Months", "6 Months", etc.
    """
    try:
        if pd.isna(duration_str) or duration_str == 'Unspecified':
            return np.nan
        
        # Convert to string and clean
        duration_str = str(duration_str).replace('"', '').replace("'", '').replace('`', '').strip()
        
        # Extract numeric part
        numeric_part = ''.join(filter(str.isdigit, duration_str))
        
        if numeric_part:
            return int(numeric_part)
        else:
            return np.nan
    except:
        return np.nan

def handle_missing_values(df):
    """
    Handle missing values in the dataframe
    """
    # Drop rows with missing essential columns
    df = df.dropna(subset=['Role', 'Company Name'])
    
    # Fill missing skills with empty lists
    df["Skills"] = df["Skills"].fillna('[]')
    
    # Fill missing perks with empty lists
    df["Perks"] = df["Perks"].fillna('[]')
    
    # Fill missing location with 'Not specified'
    df["Location"] = df["Location"].fillna('Not specified')
    
    # Fill missing Stipend with 'Unspecified'
    df["Stipend"] = df["Stipend"].fillna('Unspecified')
    
    # Convert Duration to numeric first, then fill with median
    df["Duration"] = df["Duration"].apply(extract_numeric_duration)
    duration_median = df["Duration"].median()
    df["Duration"] = df["Duration"].fillna(duration_median)
    
    # Fill applications with 0 for "Be an early applicant" or missing
    df["Number of Applications"] = df["Number of Applications"].replace('Be an early applicant', 0)
    df["Number of Applications"] = pd.to_numeric(df["Number of Applications"], errors='coerce').fillna(0)
    
    return df

# Check missing values before handling
print("Missing values before handling:")
print(df.isnull().sum())

# Handle missing values
df = handle_missing_values(df)

print("\nMissing values after handling:")
print(df.isnull().sum())

print("\nData types after processing:")
print(df.dtypes)

Missing values before handling:
Internship Id                0
Role                         0
Company Name                 0
Location                     0
Duration                     0
Stipend                      0
Intern Type                  0
Skills                      79
Perks                      403
Hiring Since                 2
Opportunity Date             1
Opening                      0
Hired Candidate           3154
Number of Applications       0
Website Link              2330
dtype: int64

Missing values after handling:
Internship Id                0
Role                         0
Company Name                 0
Location                     0
Duration                     0
Stipend                      0
Intern Type                  0
Skills                       0
Perks                        0
Hiring Since                 2
Opportunity Date             1
Opening                      0
Hired Candidate           3154
Number of Applications       0
Website Link            

In [2]:
# Continue from the previous code...

print("\n=== STEP 1.3: Data Transformation and Feature Engineering ===\n")

# 1.3.1 Convert string representations of lists to actual lists
def safe_literal_eval(value):
    """
    Safely convert string representations of lists to actual lists
    """
    try:
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            return ast.literal_eval(value)
        return value
    except (ValueError, SyntaxError):
        return value

# Apply to Skills and Perks columns
df["Skills"] = df["Skills"].apply(safe_literal_eval)
df["Perks"] = df["Perks"].apply(safe_literal_eval)

# 1.3.2 Extract additional features from existing columns
def extract_stipend_info(stipend_str):
    """
    Extract numeric stipend values and currency information
    """
    if pd.isna(stipend_str) or stipend_str == 'Unspecified':
        return np.nan, 'Unspecified'
    
    stipend_str = str(stipend_str).lower()
    
    # Check for common stipend patterns
    if 'unpaid' in stipend_str:
        return 0, 'Unpaid'
    elif 'performance' in stipend_str:
        return np.nan, 'Performance-based'
    elif 'negotiable' in stipend_str:
        return np.nan, 'Negotiable'
    
    # Extract numeric values
    import re
    numbers = re.findall(r'\d+[,.]?\d*', stipend_str)
    
    if numbers:
        # Convert to float (handle commas as thousand separators)
        numeric_value = float(numbers[0].replace(',', ''))
        
        # Detect currency
        if '₹' in stipend_str or 'inr' in stipend_str or 'rs' in stipend_str:
            currency = 'INR'
        elif '$' in stipend_str or 'usd' in stipend_str:
            currency = 'USD'
        elif '€' in stipend_str or 'eur' in stipend_str:
            currency = 'EUR'
        elif '£' in stipend_str or 'gbp' in stipend_str:
            currency = 'GBP'
        else:
            currency = 'Unknown'
        
        return numeric_value, currency
    else:
        return np.nan, 'Unspecified'

# Apply stipend extraction
stipend_info = df["Stipend"].apply(extract_stipend_info)
df["Stipend Amount"] = stipend_info.apply(lambda x: x[0])
df["Stipend Currency"] = stipend_info.apply(lambda x: x[1])

# 1.3.3 Create binary flags for important features
df["Has Skills"] = df["Skills"].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)
df["Has Perks"] = df["Perks"].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)
df["Stipend Specified"] = df["Stipend Amount"].notna()

# 1.3.4 Process Location data
def extract_city_country(location_str):
    """
    Extract city and country from location string
    """
    if pd.isna(location_str) or location_str == 'Not specified':
        return 'Unknown', 'Unknown'
    
    location_str = str(location_str).strip()
    
    # Common patterns
    if ',' in location_str:
        parts = location_str.split(',')
        city = parts[0].strip()
        country = parts[-1].strip() if len(parts) > 1 else 'Unknown'
    else:
        city = location_str
        country = 'Unknown'
    
    return city, country

location_info = df["Location"].apply(extract_city_country)
df["City"] = location_info.apply(lambda x: x[0])
df["Country"] = location_info.apply(lambda x: x[1])

# 1.3.5 Create duration categories
def categorize_duration(duration_months):
    """
    Categorize duration into meaningful groups
    """
    if pd.isna(duration_months):
        return 'Unknown'
    
    if duration_months <= 2:
        return 'Short-term (1-2 months)'
    elif duration_months <= 4:
        return 'Medium-term (3-4 months)'
    elif duration_months <= 6:
        return 'Long-term (5-6 months)'
    else:
        return 'Extended (>6 months)'

df["Duration Category"] = df["Duration"].apply(categorize_duration)

# 1.3.6 Create application volume indicator
def application_volume_indicator(app_count):
    """
    Categorize application volume
    """
    if pd.isna(app_count):
        return 'Unknown'
    
    if app_count == 0:
        return 'Early Applicant'
    elif app_count <= 10:
        return 'Low Applications'
    elif app_count <= 50:
        return 'Medium Applications'
    elif app_count <= 100:
        return 'High Applications'
    else:
        return 'Very High Applications'

df["Application Volume"] = df["Number of Applications"].apply(application_volume_indicator)

# 1.3.7 Extract posting date features (if available)
if 'Opportunity Date' in df.columns:
    try:
        df['Posting Date'] = pd.to_datetime(df['Opportunity Date'], errors='coerce')
        df['Posting Month'] = df['Posting Date'].dt.month
        df['Posting Year'] = df['Posting Date'].dt.year
        df['Days Since Posted'] = (pd.Timestamp.now() - df['Posting Date']).dt.days
    except:
        print("Could not process Opportunity Date column")

# 1.3.8 Create skill count features
df["Number of Skills"] = df["Skills"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df["Number of Perks"] = df["Perks"].apply(lambda x: len(x) if isinstance(x, list) else 0)

# 1.3.9 Display transformation summary
print("Data Transformation Summary:")
print("=" * 50)
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"New features created: {len(df.columns) - len(pd.read_csv('internship_data.csv').columns)}")

print("\nNew columns created:")
new_columns = [col for col in df.columns if col not in pd.read_csv('internship_data.csv').columns]
for col in new_columns:
    print(f"  - {col}")

print("\nSample of transformed data:")
print(df[['Role', 'Company Name', 'Duration', 'Stipend Amount', 'Stipend Currency', 
          'Duration Category', 'Application Volume']].head())

print("\nData types after transformation:")
print(df.dtypes)

# 1.3.10 Save the transformed data (optional)
df.to_csv('transformed_internship_data.csv', index=False)
print(f"\nTransformed data saved to 'transformed_internship_data.csv'")


=== STEP 1.3: Data Transformation and Feature Engineering ===



/tmp/ipykernel_4745/241086750.py:141: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Posting Date'] = pd.to_datetime(df['Opportunity Date'], errors='coerce')


Data Transformation Summary:
Total rows: 6642
Total columns: 30
New features created: 15

New columns created:
  - Stipend Amount
  - Stipend Currency
  - Has Skills
  - Has Perks
  - Stipend Specified
  - City
  - Country
  - Duration Category
  - Application Volume
  - Posting Date
  - Posting Month
  - Posting Year
  - Days Since Posted
  - Number of Skills
  - Number of Perks

Sample of transformed data:
                                         Role  \
0     Business Development (Sales) Internship   
1             Human Resources (HR) Internship   
2  Content & E-Commerce Management Internship   
3               Project Management Internship   
4                Digital Marketing Internship   

                                       Company Name  Duration  Stipend Amount  \
0                        Madbrains Technologies LLP         4          5000.0   
1                    Jobs Flash Consulting Services         6          8000.0   
2                                    Fall For Flor

In [3]:
# Continue from the previous code...

print("\n=== STEP 1.4: Data Validation and Quality Assurance ===\n")

# 4.1 Data Quality Metrics
def calculate_data_quality_metrics(df):
    """
    Calculate comprehensive data quality metrics
    """
    quality_metrics = {}
    
    # Completeness metrics
    quality_metrics['total_rows'] = len(df)
    quality_metrics['total_columns'] = len(df.columns)
    quality_metrics['total_cells'] = len(df) * len(df.columns)
    quality_metrics['null_cells'] = df.isnull().sum().sum()
    quality_metrics['completeness_rate'] = 1 - (quality_metrics['null_cells'] / quality_metrics['total_cells'])
    
    # Column-wise completeness
    column_completeness = {}
    for col in df.columns:
        null_count = df[col].isnull().sum()
        completeness = 1 - (null_count / len(df))
        column_completeness[col] = {
            'null_count': null_count,
            'completeness_rate': completeness,
            'data_type': str(df[col].dtype)
        }
    
    quality_metrics['column_analysis'] = column_completeness
    
    # Data type consistency check (handle list columns specially)
    dtype_consistency = {}
    for col in df.columns:
        if df[col].dtype == 'object':
            # Check if column contains lists
            sample_non_null = df[col].dropna()
            if len(sample_non_null) > 0 and isinstance(sample_non_null.iloc[0], list):
                # For list columns, count unique list lengths instead of unique values
                unique_lengths = sample_non_null.apply(len).nunique()
                sample_values = sample_non_null.iloc[0] if len(sample_non_null) > 0 else []
                dtype_consistency[col] = {
                    'unique_list_lengths': unique_lengths,
                    'sample_first_list': sample_values[:3],  # Show first 3 items
                    'is_list_column': True
                }
            else:
                # For regular string columns
                unique_count = df[col].nunique()
                sample_values = df[col].dropna().unique()[:3] if unique_count > 0 else []
                dtype_consistency[col] = {
                    'unique_values': unique_count,
                    'sample_values': sample_values.tolist(),
                    'is_list_column': False
                }
    
    quality_metrics['categorical_analysis'] = dtype_consistency
    
    return quality_metrics

# Calculate quality metrics
quality_metrics = calculate_data_quality_metrics(df)

print("Data Quality Report:")
print("=" * 50)
print(f"Total Rows: {quality_metrics['total_rows']}")
print(f"Total Columns: {quality_metrics['total_columns']}")
print(f"Completeness Rate: {quality_metrics['completeness_rate']:.2%}")
print(f"Null Cells: {quality_metrics['null_cells']}")

# 4.2 Validation Checks
def perform_validation_checks(df):
    """
    Perform comprehensive validation checks on the dataset
    """
    validation_results = {}
    
    # Check 1: Essential columns should not have null values
    essential_columns = ['Role', 'Company Name', 'Duration']
    for col in essential_columns:
        if col in df.columns:
            null_count = df[col].isnull().sum()
            validation_results[f'{col}_null_check'] = {
                'passed': null_count == 0,
                'null_count': null_count,
                'message': f'{col} has {null_count} null values'
            }
    
    # Check 2: Numeric ranges validation
    numeric_checks = {
        'Duration': {'min': 1, 'max': 24},
        'Number of Applications': {'min': 0, 'max': 10000},
        'Stipend Amount': {'min': 0, 'max': 100000}
    }
    
    for col, ranges in numeric_checks.items():
        if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
            valid_count = ((df[col] >= ranges['min']) & (df[col] <= ranges['max'])).sum()
            invalid_count = len(df) - valid_count - df[col].isnull().sum()
            validation_results[f'{col}_range_check'] = {
                'passed': invalid_count == 0,
                'invalid_count': invalid_count,
                'message': f'{col} has {invalid_count} values outside range [{ranges["min"]}, {ranges["max"]}]'
            }
    
    # Check 3: Data type consistency
    expected_dtypes = {
        'Duration': 'numeric',
        'Number of Applications': 'numeric',
        'Stipend Amount': 'numeric'
    }
    
    for col, expected_type in expected_dtypes.items():
        if col in df.columns:
            if expected_type == 'numeric' and pd.api.types.is_numeric_dtype(df[col]):
                validation_results[f'{col}_dtype_check'] = {'passed': True, 'message': f'{col} is numeric'}
            else:
                validation_results[f'{col}_dtype_check'] = {'passed': False, 'message': f'{col} type mismatch'}
    
    # Check 4: List columns validation
    list_columns = ['Skills', 'Perks']
    for col in list_columns:
        if col in df.columns:
            # Check if column contains lists
            sample_non_null = df[col].dropna()
            if len(sample_non_null) > 0 and isinstance(sample_non_null.iloc[0], list):
                validation_results[f'{col}_list_check'] = {
                    'passed': True,
                    'message': f'{col} contains valid lists'
                }
            else:
                validation_results[f'{col}_list_check'] = {
                    'passed': False,
                    'message': f'{col} does not contain lists'
                }
    
    # Check 5: Unique identifier check
    if 'Role' in df.columns and 'Company Name' in df.columns:
        duplicates = df.duplicated(subset=['Role', 'Company Name']).sum()
        validation_results['duplicate_check'] = {
            'passed': duplicates == 0,
            'duplicate_count': duplicates,
            'message': f'Found {duplicates} duplicate Role-Company pairs'
        }
    
    # Check 6: Cross-field validation
    if 'Stipend Amount' in df.columns and 'Stipend Currency' in df.columns:
        stipend_without_currency = ((df['Stipend Amount'].notna()) & 
                                   (df['Stipend Currency'].isin(['Unspecified', 'Unknown']))).sum()
        validation_results['stipend_consistency_check'] = {
            'passed': stipend_without_currency == 0,
            'inconsistent_count': stipend_without_currency,
            'message': f'Found {stipend_without_currency} records with stipend amount but no currency'
        }
    
    return validation_results

# Perform validation checks
validation_results = perform_validation_checks(df)

print("\nValidation Results:")
print("=" * 50)
all_passed = True
for check_name, result in validation_results.items():
    status = "PASS" if result['passed'] else "FAIL"
    print(f"{check_name}: {status} - {result['message']}")
    if not result['passed']:
        all_passed = False

print(f"\nOverall Validation: {'PASSED' if all_passed else 'FAILED'}")

# 4.3 Outlier Detection
def detect_outliers(df):
    """
    Detect outliers in numeric columns
    """
    outlier_report = {}
    
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    
    for col in numeric_columns:
        # Skip if all values are null or constant
        if df[col].nunique() <= 1:
            continue
            
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Handle case where IQR is 0 (constant data)
        if IQR == 0:
            continue
            
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = len(outliers)
        
        outlier_report[col] = {
            'outlier_count': outlier_count,
            'outlier_percentage': (outlier_count / len(df)) * 100,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'outlier_examples': outliers[col].head(3).tolist() if outlier_count > 0 else []
        }
    
    return outlier_report

# Detect outliers
outlier_report = detect_outliers(df)

print("\nOutlier Analysis:")
print("=" * 50)
for col, report in outlier_report.items():
    print(f"{col}: {report['outlier_count']} outliers ({report['outlier_percentage']:.1f}%)")
    if report['outlier_count'] > 0:
        print(f"  Range: [{report['lower_bound']:.2f}, {report['upper_bound']:.2f}]")
        print(f"  Examples: {report['outlier_examples']}")

# 4.4 Data Quality Summary
def generate_quality_summary(quality_metrics, validation_results, outlier_report):
    """
    Generate comprehensive data quality summary
    """
    summary = {
        'overall_score': 0,
        'completeness_score': quality_metrics['completeness_rate'] * 100,
        'validation_score': (sum(1 for r in validation_results.values() if r['passed']) / len(validation_results)) * 100,
        'critical_issues': [],
        'warnings': [],
        'recommendations': []
    }
    
    # Calculate overall score (weighted average)
    summary['overall_score'] = (summary['completeness_score'] * 0.4 + summary['validation_score'] * 0.6)
    
    # Identify critical issues
    for check_name, result in validation_results.items():
        if not result['passed'] and 'null' in check_name.lower():
            summary['critical_issues'].append(result['message'])
        elif not result['passed']:
            summary['warnings'].append(result['message'])
    
    # Add outlier warnings
    for col, report in outlier_report.items():
        if report['outlier_percentage'] > 5:
            summary['warnings'].append(f"High outliers in {col}: {report['outlier_percentage']:.1f}%")
    
    # Generate recommendations
    if summary['completeness_score'] < 95:
        summary['recommendations'].append("Improve data completeness for better analysis")
    
    if any(not r['passed'] for r in validation_results.values()):
        summary['recommendations'].append("Address validation failures before analysis")
    
    return summary

# Generate quality summary
quality_summary = generate_quality_summary(quality_metrics, validation_results, outlier_report)

print("\nData Quality Summary:")
print("=" * 50)
print(f"Overall Quality Score: {quality_summary['overall_score']:.1f}/100")
print(f"Completeness Score: {quality_summary['completeness_score']:.1f}/100")
print(f"Validation Score: {quality_summary['validation_score']:.1f}/100")

if quality_summary['critical_issues']:
    print("\n🚨 Critical Issues:")
    for issue in quality_summary['critical_issues']:
        print(f"  • {issue}")

if quality_summary['warnings']:
    print("\n⚠️  Warnings:")
    for warning in quality_summary['warnings']:
        print(f"  • {warning}")

if quality_summary['recommendations']:
    print("\n💡 Recommendations:")
    for recommendation in quality_summary['recommendations']:
        print(f"  • {recommendation}")

# 4.5 Save validation report
validation_df = pd.DataFrame([
    {
        'check_name': name,
        'status': 'PASS' if result['passed'] else 'FAIL',
        'message': result['message']
    }
    for name, result in validation_results.items()
])

validation_df.to_csv('data_validation_report.csv', index=False)
print(f"\nValidation report saved to 'data_validation_report.csv'")

print("\n✅ Step 1.4 completed successfully! Data is ready for analysis.")


=== STEP 1.4: Data Validation and Quality Assurance ===



Data Quality Report:
Total Rows: 6642
Total Columns: 30
Completeness Rate: 83.90%
Null Cells: 32087

Validation Results:
Role_null_check: PASS - Role has 0 null values
Company Name_null_check: PASS - Company Name has 0 null values
Duration_null_check: PASS - Duration has 0 null values
Duration_range_check: FAIL - Duration has 6 values outside range [1, 24]
Number of Applications_range_check: PASS - Number of Applications has 0 values outside range [0, 10000]
Stipend Amount_range_check: PASS - Stipend Amount has 0 values outside range [0, 100000]
Duration_dtype_check: PASS - Duration is numeric
Number of Applications_dtype_check: PASS - Number of Applications is numeric
Stipend Amount_dtype_check: PASS - Stipend Amount is numeric
Skills_list_check: PASS - Skills contains valid lists
Perks_list_check: PASS - Perks contains valid lists
duplicate_check: FAIL - Found 172 duplicate Role-Company pairs
stipend_consistency_check: FAIL - Found 50 records with stipend amount but no currency

Over

In [4]:
# Continue from the previous code...

print("\n=== STEP 1.5: Data Export and Documentation ===\n")

# 5.1 Final Data Quality Check
print("Final Data Quality Check:")
print("=" * 50)
print(f"Dataset Shape: {df.shape}")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Check data types
print("\nFinal Data Types:")
for col, dtype in df.dtypes.items():
    print(f"  {col}: {dtype}")

# 5.2 Create comprehensive dataset documentation
def convert_to_serializable(obj):
    """
    Convert NumPy types and other non-serializable objects to JSON-serializable types
    """
    if isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, (np.bool_)):
        return bool(obj)
    elif isinstance(obj, (np.ndarray)):
        return obj.tolist()
    elif isinstance(obj, (pd.Timestamp)):
        return obj.isoformat()
    elif isinstance(obj, (pd.Timedelta)):
        return str(obj)
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    else:
        return obj

def create_dataset_documentation(df, quality_metrics, validation_results):
    """
    Create comprehensive documentation for the processed dataset
    """
    documentation = {
        'overview': {
            'dataset_name': 'Processed Internship Data',
            'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'total_records': len(df),
            'total_features': len(df.columns),
            'data_quality_score': convert_to_serializable(quality_metrics.get('completeness_rate', 0) * 100)
        },
        'column_descriptions': {},
        'processing_steps': [
            '1.1 - Data Loading and Initial Inspection',
            '1.2 - Missing Value Handling',
            '1.3 - Data Transformation and Feature Engineering',
            '1.4 - Data Validation and Quality Assurance',
            '1.5 - Data Export and Documentation'
        ],
        'quality_metrics': convert_to_serializable(quality_metrics),
        'validation_results': convert_to_serializable(validation_results)
    }
    
    # Column descriptions
    for col in df.columns:
        documentation['column_descriptions'][col] = {
            'data_type': str(df[col].dtype),
            'null_count': convert_to_serializable(df[col].isnull().sum()),
            'unique_values': convert_to_serializable(df[col].nunique() if df[col].dtype != 'object' or not any(isinstance(x, list) for x in df[col].dropna()) else 'List data'),
            'description': get_column_description(col)
        }
    
    return documentation

def get_column_description(column_name):
    """
    Get description for each column
    """
    descriptions = {
        'Internship Id': 'Unique identifier for the internship',
        'Role': 'Position title of the internship',
        'Company Name': 'Name of the company offering the internship',
        'Location': 'Original location information',
        'Duration': 'Duration of internship in months (numeric)',
        'Stipend': 'Original stipend information as string',
        'Intern Type': 'Type of internship (e.g., Full-time, Part-time)',
        'Skills': 'List of required skills for the internship',
        'Perks': 'List of perks offered with the internship',
        'Hiring Since': 'How long the company has been hiring',
        'Opportunity Date': 'Date when opportunity was posted',
        'Opening': 'Number of open positions',
        'Hired Candidate': 'Information about hired candidates',
        'Number of Applications': 'Count of applications received',
        'Website Link': 'URL to apply for the internship',
        'Stipend Amount': 'Extracted numeric stipend amount',
        'Stipend Currency': 'Currency of the stipend amount',
        'Has Skills': 'Boolean indicating if skills are specified',
        'Has Perks': 'Boolean indicating if perks are specified',
        'Stipend Specified': 'Boolean indicating if stipend is specified',
        'City': 'Extracted city from location',
        'Country': 'Extracted country from location',
        'Duration Category': 'Categorized duration (Short-term, Medium-term, Long-term, Extended)',
        'Application Volume': 'Categorized application volume',
        'Posting Date': 'Date when internship was posted',
        'Posting Month': 'Month when internship was posted',
        'Posting Year': 'Year when internship was posted',
        'Days Since Posted': 'Number of days since internship was posted',
        'Number of Skills': 'Count of skills required',
        'Number of Perks': 'Count of perks offered'
    }
    
    return descriptions.get(column_name, 'No description available')

# Create documentation
dataset_docs = create_dataset_documentation(df, quality_metrics, validation_results)

# 5.3 Export the processed data in multiple formats
print("\nExporting processed data...")

# Export to CSV
csv_filename = 'processed_internship_data.csv'
df.to_csv(csv_filename, index=False)
print(f"✓ CSV file saved: {csv_filename}")

# Export to Excel (if xlsxwriter is available)
try:
    import xlsxwriter
    excel_filename = 'processed_internship_data.xlsx'
    with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Internship Data', index=False)
        
        # Add summary sheet
        summary_data = []
        for col in df.columns:
            summary_data.append({
                'Column': col,
                'Data Type': str(df[col].dtype),
                'Null Count': int(df[col].isnull().sum()),
                'Unique Values': int(df[col].nunique()) if df[col].dtype != 'object' or not any(isinstance(x, list) for x in df[col].dropna()) else 'List data'
            })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Data Summary', index=False)
    
    print(f"✓ Excel file saved: {excel_filename}")
except ImportError:
    print("⚠️  xlsxwriter not installed, skipping Excel export")

# Export to JSON (for web applications)
json_filename = 'processed_internship_data.json'

# Convert numpy types to native Python types before JSON export
def convert_df_to_serializable(df):
    """Convert DataFrame with numpy types to serializable format"""
    result = df.to_dict(orient='records')
    return [convert_to_serializable(record) for record in result]

serializable_data = convert_df_to_serializable(df)

# with open(json_filename, 'w') as f:
#     json.dump(serializable_data, f, indent=2, default=convert_to_serializable)
def safe_json_serialize(obj):
    """Safely serialize objects for JSON"""
    if isinstance(obj, (np.integer, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.bool_)):
        return bool(obj)
    elif isinstance(obj, (pd.Timestamp)):
        return obj.isoformat()
    elif isinstance(obj, (list, tuple)):
        return [safe_json_serialize(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: safe_json_serialize(value) for key, value in obj.items()}
    elif hasattr(obj, 'dtype'):  # numpy arrays
        return obj.tolist()
    else:
        return str(obj)  # Fallback to string representation
df_serializable = df.applymap(safe_json_serialize)
df_serializable.to_json(json_filename, orient='records', indent=2)
print(f"✓ JSON file saved: {json_filename}")

# 5.4 Save documentation files
# Save documentation as JSON
docs_filename = 'dataset_documentation.json'
with open(docs_filename, 'w') as f:
    json.dump(dataset_docs, f, indent=2, default=convert_to_serializable)
print(f"✓ Documentation saved: {docs_filename}")

# Save documentation as Markdown
md_filename = 'DATASET_README.md'
with open(md_filename, 'w') as f:
    f.write(f"# Processed Internship Dataset Documentation\n\n")
    f.write(f"## Overview\n")
    f.write(f"- **Processing Date**: {dataset_docs['overview']['processing_date']}\n")
    f.write(f"- **Total Records**: {dataset_docs['overview']['total_records']:,}\n")
    f.write(f"- **Total Features**: {dataset_docs['overview']['total_features']}\n")
    f.write(f"- **Data Quality Score**: {dataset_docs['overview']['data_quality_score']:.1f}%\n\n")
    
    f.write(f"## Processing Steps\n")
    for step in dataset_docs['processing_steps']:
        f.write(f"1. {step}\n")
    f.write(f"\n")
    
    f.write(f"## Column Descriptions\n")
    f.write(f"| Column | Data Type | Null Count | Unique Values | Description |\n")
    f.write(f"|--------|-----------|------------|---------------|-------------|\n")
    
    for col, info in dataset_docs['column_descriptions'].items():
        f.write(f"| {col} | {info['data_type']} | {info['null_count']} | {info['unique_values']} | {info['description']} |\n")
    
    f.write(f"\n## Quality Metrics\n")
    f.write(f"- **Completeness Rate**: {quality_metrics['completeness_rate']:.2%}\n")
    f.write(f"- **Null Cells**: {quality_metrics['null_cells']}\n")
    f.write(f"- **Total Cells**: {quality_metrics['total_cells']}\n")
    
    f.write(f"\n## Validation Results Summary\n")
    passed_checks = sum(1 for r in validation_results.values() if r['passed'])
    total_checks = len(validation_results)
    f.write(f"- **Passed Checks**: {passed_checks}/{total_checks} ({passed_checks/total_checks:.1%})\n")
    
    f.write(f"\n## Files Generated\n")
    f.write(f"- `{csv_filename}`: Main dataset in CSV format\n")
    f.write(f"- `{json_filename}`: Dataset in JSON format for web applications\n")
    f.write(f"- `{docs_filename}`: Comprehensive documentation in JSON format\n")
    f.write(f"- `data_validation_report.csv`: Detailed validation results\n")

print(f"✓ Markdown documentation saved: {md_filename}")

# 5.5 Create sample analysis-ready datasets
print("\nCreating sample analysis datasets...")

# Sample 1: Numeric analysis dataset
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = ['Role', 'Company Name', 'Duration Category', 'Stipend Currency', 
                   'Application Volume', 'City', 'Country']

analysis_df = df[numeric_cols + categorical_cols].copy()
analysis_df.to_csv('analysis_ready_dataset.csv', index=False)
print(f"✓ Analysis-ready dataset saved: analysis_ready_dataset.csv")

# Sample 2: Machine learning ready dataset (encoded categorical variables)
ml_ready_df = df.copy()
# One-hot encode categorical variables
categorical_to_encode = ['Duration Category', 'Stipend Currency', 'Application Volume']
for col in categorical_to_encode:
    if col in ml_ready_df.columns:
        dummies = pd.get_dummies(ml_ready_df[col], prefix=col, drop_first=True)
        ml_ready_df = pd.concat([ml_ready_df, dummies], axis=1)
        ml_ready_df.drop(col, axis=1, inplace=True)

ml_ready_df.to_csv('ml_ready_dataset.csv', index=False)
print(f"✓ ML-ready dataset saved: ml_ready_dataset.csv")

# 5.6 Final summary report
print("\n" + "="*60)
print("DATA PROCESSING PIPELINE COMPLETED SUCCESSFULLY!")
print("="*60)

print(f"\n📊 Final Dataset Statistics:")
print(f"   Records: {len(df):,}")
print(f"   Features: {len(df.columns)}")
print(f"   Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   Quality Score: {dataset_docs['overview']['data_quality_score']:.1f}%")

print(f"\n💾 Files Created:")
print(f"   {csv_filename} - Main processed dataset")
print(f"   {json_filename} - JSON format for applications")
print(f"   {docs_filename} - Comprehensive documentation")
print(f"   {md_filename} - Readme file")
print(f"   data_validation_report.csv - Validation results")
print(f"   analysis_ready_dataset.csv - Analysis optimized")
print(f"   ml_ready_dataset.csv - Machine learning ready")

print(f"\n✅ Processing Steps Completed:")
for i, step in enumerate(dataset_docs['processing_steps'], 1):
    print(f"   {i}. {step}")

print(f"\n🎯 Dataset is now ready for:")
print(f"   • Exploratory Data Analysis (EDA)")
print(f"   • Statistical Analysis")
print(f"   • Machine Learning Modeling")
print(f"   • Visualization and Reporting")

print(f"\n📋 Next Steps:")
print(f"   1. Perform exploratory data analysis")
print(f"   2. Create visualizations and dashboards")
print(f"   3. Build predictive models")
print(f"   4. Generate insights and recommendations")

print(f"\n{'-'*60}")
print("Data preprocessing pipeline completed at:", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('-'*60)


=== STEP 1.5: Data Export and Documentation ===

Final Data Quality Check:
Dataset Shape: (6642, 30)


Memory Usage: 8.64 MB

Final Data Types:
  Internship Id: int64
  Role: object
  Company Name: object
  Location: object
  Duration: int64
  Stipend: object
  Intern Type: object
  Skills: object
  Perks: object
  Hiring Since: object
  Opportunity Date: object
  Opening: int64
  Hired Candidate: object
  Number of Applications: float64
  Website Link: object
  Stipend Amount: float64
  Stipend Currency: object
  Has Skills: bool
  Has Perks: bool
  Stipend Specified: bool
  City: object
  Country: object
  Duration Category: object
  Application Volume: object
  Posting Date: datetime64[ns]
  Posting Month: float64
  Posting Year: float64
  Days Since Posted: float64
  Number of Skills: int64
  Number of Perks: int64

Exporting processed data...
✓ CSV file saved: processed_internship_data.csv
⚠️  xlsxwriter not installed, skipping Excel export


/tmp/ipykernel_4745/81956912.py:180: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_serializable = df.applymap(safe_json_serialize)


✓ JSON file saved: processed_internship_data.json


NameError: name 'json' is not defined

In [ ]:
# Continue from the previous code...

print("\n=== STEP 3: Recommendation Engine Implementation ===\n")

# 3.1 Load the processed data
print("Loading processed data for recommendation engine...")
df_recommend = pd.read_csv('processed_internship_data.csv')

# Convert string representations back to lists for Skills and Perks
def safe_string_to_list(value):
    """Safely convert string representation to list"""
    if pd.isna(value) or value == '[]':
        return []
    try:
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            return ast.literal_eval(value)
        return value
    except:
        return []

df_recommend['Skills'] = df_recommend['Skills'].apply(safe_string_to_list)
df_recommend['Perks'] = df_recommend['Perks'].apply(safe_string_to_list)

print(f"Loaded {len(df_recommend)} internships for recommendation")

# 3.2 Feature Engineering for Recommendation
print("\n3.2 Preparing features for recommendation...")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import re

def preprocess_text(text):
    """Clean and preprocess text for vectorization"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)      # Remove extra spaces
    return text.strip()

def create_feature_vectors(df):
    """
    Create feature vectors for content-based filtering with consistent dimensions
    """
    # 3.2.1 Text Features - TF-IDF Vectorization
    print("  - Vectorizing text features...")
    
    # Combine text features
    df['combined_text'] = (
        df['Role'].apply(preprocess_text) + ' ' +
        df['Company Name'].apply(preprocess_text) + ' ' +
        df['Skills'].apply(lambda x: ' '.join([preprocess_text(skill) for skill in x]) if isinstance(x, list) else '') + ' ' +
        df['Perks'].apply(lambda x: ' '.join([preprocess_text(perk) for perk in x]) if isinstance(x, list) else '') + ' ' +
        df['Location'].apply(preprocess_text)
    )
    
    # TF-IDF for combined text - fix vocabulary size for consistency
    tfidf = TfidfVectorizer(
        max_features=500,  # Reduced to ensure consistency
        stop_words='english',
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.9
    )
    
    tfidf_matrix = tfidf.fit_transform(df['combined_text'])
    print(f"  - TF-IDF matrix shape: {tfidf_matrix.shape}")
    
    # 3.2.2 Numerical Features - Standardization
    print("  - Standardizing numerical features...")
    
    numerical_features = [
        'Duration', 'Stipend Amount', 'Number of Applications',
        'Number of Skills', 'Number of Perks', 'Days Since Posted'
    ]
    
    # Filter out features that might not exist
    available_numerical = [f for f in numerical_features if f in df.columns]
    numerical_data = df[available_numerical].fillna(0)
    
    scaler = StandardScaler()
    numerical_matrix = scaler.fit_transform(numerical_data)
    print(f"  - Numerical matrix shape: {numerical_matrix.shape}")
    
    # 3.2.3 Categorical Features - One-hot encoding with fixed categories
    print("  - Encoding categorical features...")
    
    categorical_features = [
        'Duration Category', 'Stipend Currency', 'Application Volume',
        'City', 'Country', 'Intern Type'
    ]
    
    available_categorical = [f for f in categorical_features if f in df.columns]
    
    # Get all possible categories for consistent encoding
    categorical_dummies_list = []
    for col in available_categorical:
        # Get all unique values in the column
        unique_vals = df[col].fillna('Unknown').unique()
        # Create dummy variables manually to ensure consistency
        for val in unique_vals:
            col_name = f"{col}_{val}"
            categorical_dummies_list.append((col, val, col_name))
    
    # Create consistent categorical matrix
    categorical_matrix = np.zeros((len(df), len(categorical_dummies_list)))
    
    for i, (col, val, col_name) in enumerate(categorical_dummies_list):
        categorical_matrix[:, i] = (df[col].fillna('Unknown') == val).astype(int)
    
    print(f"  - Categorical matrix shape: {categorical_matrix.shape}")
    
    # 3.2.4 Combine all feature matrices
    print("  - Combining all feature matrices...")
    
    from scipy.sparse import hstack, csr_matrix
    
    # Convert to sparse matrices for efficient combination
    numerical_sparse = csr_matrix(numerical_matrix)
    categorical_sparse = csr_matrix(categorical_matrix)
    
    combined_matrix = hstack([tfidf_matrix, numerical_sparse, categorical_sparse])
    print(f"  - Final combined matrix shape: {combined_matrix.shape}")
    
    # Store feature information for consistent transformation
    feature_info = {
        'tfidf': tfidf,
        'scaler': scaler,
        'numerical_columns': available_numerical,
        'categorical_mapping': categorical_dummies_list,
        'categorical_columns': available_categorical,
        'feature_dimensions': combined_matrix.shape[1],
        'tfidf_dimensions': tfidf_matrix.shape[1],
        'numerical_dimensions': numerical_matrix.shape[1],
        'categorical_dimensions': categorical_matrix.shape[1]
    }
    
    return combined_matrix, feature_info

# Create feature vectors
feature_matrix, feature_info = create_feature_vectors(df_recommend)

# 3.3 Build Robust Recommendation Functions
print("\n3.3 Building robust recommendation functions...")

class InternshipRecommender:
    def __init__(self, df, feature_matrix, feature_info):
        self.df = df.reset_index(drop=True)
        self.feature_matrix = feature_matrix
        self.feature_info = feature_info
        self.internship_ids = df['Internship Id'].values
        
    def _create_query_vector(self, skills_text):
        """Create a query vector with consistent dimensions"""
        # Transform skills using TF-IDF
        query_tfidf = self.feature_info['tfidf'].transform([skills_text])
        
        # Create numerical part (zeros since we don't have numerical data for query)
        numerical_part = np.zeros((1, len(self.feature_info['numerical_columns'])))
        
        # Create categorical part (zeros since we don't have categorical data for query)
        categorical_part = np.zeros((1, len(self.feature_info['categorical_mapping'])))
        
        # Combine all parts
        from scipy.sparse import hstack, csr_matrix
        
        query_vector = hstack([
            query_tfidf,
            csr_matrix(numerical_part),
            csr_matrix(categorical_part)
        ])
        
        return query_vector
    
    def recommend_by_skills(self, skills, top_n=5):
        """Recommend internships based on skills match"""
        print(f"  - Finding recommendations for skills: {skills}")
        
        if not skills:
            return []
        
        # Create query text
        skills_text = ' '.join([preprocess_text(skill) for skill in skills])
        
        try:
            # Create query vector with consistent dimensions
            query_vector = self._create_query_vector(skills_text)
            
            # Calculate similarity using only TF-IDF part for skill matching
            tfidf_similarities = cosine_similarity(
                query_vector[:, :self.feature_info['tfidf_dimensions']],
                self.feature_matrix[:, :self.feature_info['tfidf_dimensions']]
            ).flatten()
            
            # Get top recommendations
            top_indices = tfidf_similarities.argsort()[-top_n:][::-1]
            
            recommendations = []
            for idx in top_indices:
                internship = self.df.iloc[idx]
                recommendations.append({
                    'internship_id': int(internship['Internship Id']),
                    'role': str(internship['Role']),
                    'company': str(internship['Company Name']),
                    'similarity_score': float(tfidf_similarities[idx]),
                    'skills': internship['Skills'][:5],  # Limit to 5 skills
                    'stipend': str(internship['Stipend']),
                    'location': str(internship['Location']),
                    'duration': int(internship['Duration']) if not pd.isna(internship['Duration']) else None
                })
            
            return recommendations
            
        except Exception as e:
            print(f"Error in skill-based recommendation: {e}")
            return []
    
    def recommend_by_internship(self, internship_id, top_n=5):
        """Recommend similar internships based on a given internship"""
        print(f"  - Finding similar internships to ID: {internship_id}")
        
        # Find the internship index
        internship_idx = self.df[self.df['Internship Id'] == internship_id].index
        if len(internship_idx) == 0:
            print(f"Internship ID {internship_id} not found")
            return []
        
        internship_idx = internship_idx[0]
        
        try:
            # Calculate similarity to all other internships
            similarities = cosine_similarity(
                self.feature_matrix[internship_idx:internship_idx+1], 
                self.feature_matrix
            ).flatten()
            
            # Get top recommendations (excluding the input internship itself)
            top_indices = similarities.argsort()[-(top_n+1):][::-1]
            top_indices = [idx for idx in top_indices if idx != internship_idx][:top_n]
            
            recommendations = []
            for idx in top_indices:
                internship = self.df.iloc[idx]
                recommendations.append({
                    'internship_id': int(internship['Internship Id']),
                    'role': str(internship['Role']),
                    'company': str(internship['Company Name']),
                    'similarity_score': float(similarities[idx]),
                    'skills': internship['Skills'][:5],
                    'stipend': str(internship['Stipend']),
                    'location': str(internship['Location'])
                })
            
            return recommendations
            
        except Exception as e:
            print(f"Error in similar internship recommendation: {e}")
            return []
    
    def recommend_hybrid(self, skills=None, preferred_companies=None, 
                        min_stipend=0, max_duration=12, top_n=5):
        """Hybrid recommendation considering multiple factors"""
        print("  - Generating hybrid recommendations...")
        
        try:
            # Start with skill similarity if skills provided
            if skills:
                skills_text = ' '.join([preprocess_text(skill) for skill in skills])
                query_vector = self._create_query_vector(skills_text)
                skill_similarities = cosine_similarity(
                    query_vector[:, :self.feature_info['tfidf_dimensions']],
                    self.feature_matrix[:, :self.feature_info['tfidf_dimensions']]
                ).flatten()
            else:
                skill_similarities = np.ones(len(self.df))
            
            # Company preference
            if preferred_companies:
                company_mask = self.df['Company Name'].isin(preferred_companies)
                company_scores = company_mask.astype(float)
            else:
                company_scores = np.ones(len(self.df))
            
            # Stipend filter
            stipend_scores = np.where(
                self.df['Stipend Amount'].fillna(0) >= min_stipend, 1.0, 0.5
            )
            
            # Duration filter
            duration_scores = np.where(
                self.df['Duration'].fillna(0) <= max_duration, 1.0, 0.7
            )
            
            # Combine scores
            combined_scores = (
                skill_similarities * 0.4 +  # Skill match weight
                company_scores * 0.3 +      # Company preference weight
                stipend_scores * 0.2 +      # Stipend weight
                duration_scores * 0.1       # Duration weight
            )
            
            # Get top recommendations
            top_indices = combined_scores.argsort()[-top_n:][::-1]
            
            recommendations = []
            for idx in top_indices:
                internship = self.df.iloc[idx]
                recommendations.append({
                    'internship_id': int(internship['Internship Id']),
                    'role': str(internship['Role']),
                    'company': str(internship['Company Name']),
                    'composite_score': float(combined_scores[idx]),
                    'skills': internship['Skills'][:5],
                    'stipend': str(internship['Stipend']),
                    'stipend_amount': float(internship['Stipend Amount']) if not pd.isna(internship['Stipend Amount']) else None,
                    'duration': int(internship['Duration']) if not pd.isna(internship['Duration']) else None,
                    'location': str(internship['Location'])
                })
            
            return recommendations
            
        except Exception as e:
            print(f"Error in hybrid recommendation: {e}")
            return []

# Initialize the recommender
recommender = InternshipRecommender(df_recommend, feature_matrix, feature_info)
print("✓ Recommender system initialized successfully!")

# 3.4 Test the Recommendation System
print("\n3.4 Testing recommendation system...")

# Test 1: Recommend by skills
print("\nTesting skill-based recommendations:")
test_skills = ["python", "machine learning", "data analysis"]
skill_recommendations = recommender.recommend_by_skills(test_skills, top_n=3)

print(f"Found {len(skill_recommendations)} recommendations")
for i, rec in enumerate(skill_recommendations, 1):
    print(f"  {i}. {rec['role']} at {rec['company']} (Score: {rec['similarity_score']:.3f})")
    print(f"     Skills: {rec['skills'][:3]}...")
    print(f"     Stipend: {rec['stipend']}")

# Test 2: Recommend similar to a specific internship
print("\nTesting similar internship recommendations:")
if len(df_recommend) > 0:
    sample_internship_id = df_recommend['Internship Id'].iloc[0]
    print(f"Using sample internship ID: {sample_internship_id}")
    similar_recommendations = recommender.recommend_by_internship(sample_internship_id, top_n=3)
    
    print(f"Found {len(similar_recommendations)} similar internships")
    for i, rec in enumerate(similar_recommendations, 1):
        print(f"  {i}. {rec['role']} at {rec['company']} (Score: {rec['similarity_score']:.3f})")

# Test 3: Hybrid recommendations
print("\nTesting hybrid recommendations:")
hybrid_recommendations = recommender.recommend_hybrid(
    skills=["python", "web development"],
    preferred_companies=df_recommend['Company Name'].unique()[:2].tolist(),
    min_stipend=5000,
    max_duration=6,
    top_n=3
)

print(f"Found {len(hybrid_recommendations)} hybrid recommendations")
for i, rec in enumerate(hybrid_recommendations, 1):
    print(f"  {i}. {rec['role']} at {rec['company']} (Score: {rec['composite_score']:.3f})")
    if rec['stipend_amount']:
        print(f"     Stipend: ₹{rec['stipend_amount']:,.0f} | Duration: {rec['duration']} months")

# 3.5 Save the Recommender System
print("\n3.5 Saving recommender system...")

import joblib

# Save the recommender and feature info
recommender_data = {
    'recommender': recommender,
    'feature_info': feature_info,
    'df_columns': df_recommend.columns.tolist()
}

# Save using joblib
joblib.dump(recommender_data, 'internship_recommender.pkl')
print("✓ Recommender system saved to 'internship_recommender.pkl'")

# Save metadata
recommender_metadata = {
    'total_internships': len(df_recommend),
    'feature_dimensions': feature_matrix.shape[1],
    'last_trained': datetime.now().isoformat(),
    'skill_examples': list(df_recommend['Skills'].explode().value_counts().head(10).index)
}

with open('recommender_metadata.json', 'w') as f:
    json.dump(recommender_metadata, f, indent=2, default=str)

print("✓ Recommender metadata saved")

print("\n" + "="*60)
print("RECOMMENDATION ENGINE IMPLEMENTED SUCCESSFULLY!")
print("="*60)
print("✓ Fixed dimension consistency issues")
print("✓ Robust error handling implemented")
print("✓ Multiple recommendation strategies working")
print("✓ Tested with real data")
print("✓ Ready for production integration")


=== STEP 3: Recommendation Engine Implementation ===

Loading processed data for recommendation engine...
Loaded 6642 internships for recommendation

3.2 Preparing features for recommendation...
  - Vectorizing text features...
  - TF-IDF matrix shape: (6642, 500)
  - Standardizing numerical features...
  - Numerical matrix shape: (6642, 6)
  - Encoding categorical features...
  - Categorical matrix shape: (6642, 303)
  - Combining all feature matrices...
  - Final combined matrix shape: (6642, 809)

3.3 Building robust recommendation functions...
✓ Recommender system initialized successfully!

3.4 Testing recommendation system...

Testing skill-based recommendations:
  - Finding recommendations for skills: ['python', 'machine learning', 'data analysis']
Found 3 recommendations
  1. Machine Learning Internship at Climate Connect Digital (Score: 0.857)
     Skills: ['Data Science', 'Deep Learning', 'Machine Learning']...
     Stipend: ₹ 10,000-15,000 /month
  2. Machine Learning Intern

In [ ]:
# Continue from the previous code...

# print("\n=== STEP 4: Complete Recommendation Function & API Integration ===\n")

# # 4.1 Load the trained recommender
# print("4.1 Loading trained recommender system...")

# def load_recommender_system():
#     """Load the complete recommender system with error handling"""
#     try:
#         recommender_data = joblib.load('internship_recommender.pkl')
#         print("Recommender system loaded successfully")
#         return recommender_data['recommender']
#     except FileNotFoundError:
#         print("Recommender file not found. Please train the system first.")
#         return None
#     except Exception as e:
#         print(f"Error loading recommender: {e}")
#         return None

# recommender = load_recommender_system()

# if recommender is None:
#     print("Please run the training step first.")
# else:
#     print(f"Loaded recommender with {len(recommender.df)} internships")

# # 4.2 Build comprehensive recommendation functions
# print("\n4.2 Building comprehensive recommendation functions...")

# class AdvancedInternshipRecommender:
#     def __init__(self, base_recommender):
#         self.recommender = base_recommender
#         self.df = base_recommender.df
        
#     def recommend_for_user(self, user_profile, top_n=10):
#         """
#         Comprehensive recommendation based on user profile
#         """
#         print(f"Generating recommendations for user profile...")
        
#         # Start with skill-based recommendations
#         skills = user_profile.get('skills', [])
#         base_recommendations = self.recommender.recommend_by_skills(skills, top_n * 2)
        
#         if not base_recommendations:
#             base_recommendations = self.recommender.recommend_hybrid(
#                 skills=skills,
#                 preferred_companies=user_profile.get('preferred_companies'),
#                 min_stipend=user_profile.get('min_stipend', 0),
#                 max_duration=user_profile.get('max_duration', 12),
#                 top_n=top_n * 2
#             )
        
#         # Apply additional filters
#         filtered_recommendations = []
#         for rec in base_recommendations:
#             internship = self.df[self.df['Internship Id'] == rec['internship_id']].iloc[0]
            
#             # Location filter
#             preferred_locations = user_profile.get('preferred_locations', [])
#             if preferred_locations:
#                 location_match = any(loc.lower() in str(internship['Location']).lower() 
#                                    for loc in preferred_locations)
#                 if not location_match:
#                     continue
            
#             # Intern type filter
#             intern_types = user_profile.get('intern_types', [])
#             if intern_types and 'Intern Type' in internship:
#                 type_match = any(it.lower() in str(internship['Intern Type']).lower() 
#                                for it in intern_types)
#                 if not type_match:
#                     continue
            
#             # Experience level filter
#             experience_level = user_profile.get('experience_level', '')
#             if experience_level:
#                 skills_list = internship['Skills'] if isinstance(internship['Skills'], list) else []
#                 skill_count = len(skills_list)
                
#                 if experience_level == 'beginner' and skill_count > 5:
#                     continue
#                 elif experience_level == 'advanced' and skill_count < 3:
#                     continue
            
#             filtered_recommendations.append(rec)
            
#             if len(filtered_recommendations) >= top_n:
#                 break
        
#         return filtered_recommendations[:top_n]
    
#     def recommend_with_diversity(self, skills, top_n=10):
#         """Ensure diverse recommendations across companies and roles"""
#         print(f"Generating diverse recommendations...")
        
#         recommendations = self.recommender.recommend_by_skills(skills, top_n * 3)
        
#         if not recommendations:
#             return []
        
#         # Group by company and role
#         company_groups = {}
#         role_groups = {}
        
#         for rec in recommendations:
#             company = rec['company']
#             role = rec['role']
            
#             if company not in company_groups:
#                 company_groups[company] = []
#             company_groups[company].append(rec)
            
#             if role not in role_groups:
#                 role_groups[role] = []
#             role_groups[role].append(rec)
        
#         # Select diverse recommendations
#         diverse_recommendations = []
#         companies_used = set()
#         roles_used = set()
        
#         for rec in sorted(recommendations, key=lambda x: x['similarity_score'], reverse=True):
#             if len(diverse_recommendations) >= top_n:
#                 break
            
#             company = rec['company']
#             role = rec['role']
            
#             company_count = sum(1 for r in diverse_recommendations if r['company'] == company)
#             role_count = sum(1 for r in diverse_recommendations if r['role'] == role)
            
#             if company_count < 2 and role_count < 2:
#                 diverse_recommendations.append(rec)
#                 companies_used.add(company)
#                 roles_used.add(role)
        
#         if len(diverse_recommendations) < top_n:
#             for company in company_groups:
#                 if company not in companies_used:
#                     company_recs = company_groups[company]
#                     if company_recs:
#                         diverse_recommendations.append(company_recs[0])
#                         companies_used.add(company)
#                         if len(diverse_recommendations) >= top_n:
#                             break
        
#         return diverse_recommendations[:top_n]
    
#     def get_internship_details(self, internship_id):
#         """Get complete details for a specific internship"""
#         internship = self.df[self.df['Internship Id'] == internship_id]
#         if len(internship) == 0:
#             return None
        
#         internship = internship.iloc[0]
#         return {
#             'internship_id': int(internship['Internship Id']),
#             'role': str(internship['Role']),
#             'company': str(internship['Company Name']),
#             'location': str(internship['Location']),
#             'duration': int(internship['Duration']) if not pd.isna(internship['Duration']) else None,
#             'stipend': str(internship['Stipend']),
#             'stipend_amount': float(internship['Stipend Amount']) if not pd.isna(internship['Stipend Amount']) else None,
#             'skills': internship['Skills'] if isinstance(internship['Skills'], list) else [],
#             'perks': internship['Perks'] if isinstance(internship['Perks'], list) else [],
#             'applications': int(internship['Number of Applications']) if not pd.isna(internship['Number of Applications']) else None,
#         }
    
#     def search_internships(self, query, filters=None, top_n=20):
#         """Search internships with filters"""
#         print(f"Searching internships for: {query}")
        
#         if filters is None:
#             filters = {}
        
#         query = query.lower()
#         results = []
        
#         for _, internship in self.df.iterrows():
#             match_score = 0
            
#             search_fields = ['Role', 'Company Name', 'Location', 'Skills', 'Perks']
#             for field in search_fields:
#                 if field in internship:
#                     field_value = str(internship[field]).lower()
#                     if query in field_value:
#                         match_score += 1
            
#             passes_filters = True
            
#             if 'location' in filters and filters['location']:
#                 location_match = any(loc.lower() in str(internship.get('Location', '')).lower() 
#                                    for loc in filters['location'])
#                 if not location_match:
#                     passes_filters = False
            
#             if 'min_stipend' in filters and filters['min_stipend']:
#                 stipend = internship.get('Stipend Amount', 0)
#                 if pd.isna(stipend) or stipend < filters['min_stipend']:
#                     passes_filters = False
            
#             if 'max_duration' in filters and filters['max_duration']:
#                 duration = internship.get('Duration', 0)
#                 if not pd.isna(duration) and duration > filters['max_duration']:
#                     passes_filters = False
            
#             if 'companies' in filters and filters['companies']:
#                 company = str(internship.get('Company Name', '')).lower()
#                 company_match = any(comp.lower() in company for comp in filters['companies'])
#                 if not company_match:
#                     passes_filters = False
            
#             if match_score > 0 and passes_filters:
#                 results.append({
#                     'internship_id': int(internship['Internship Id']),
#                     'role': str(internship['Role']),
#                     'company': str(internship['Company Name']),
#                     'location': str(internship['Location']),
#                     'match_score': match_score,
#                     'stipend': str(internship['Stipend']),
#                     'duration': int(internship['Duration']) if not pd.isna(internship['Duration']) else None
#                 })
        
#         results.sort(key=lambda x: x['match_score'], reverse=True)
#         return results[:top_n]

# # Initialize advanced recommender
# advanced_recommender = AdvancedInternshipRecommender(recommender)
# print("Advanced recommender initialized")

# # 4.3 Test advanced recommendation functions
# print("\n4.3 Testing advanced recommendation functions...")

# # Test recommendations
# user_profile = {
#     'skills': ['python', 'machine learning', 'data analysis'],
#     'preferred_locations': ['Remote', 'Bangalore'],
#     'min_stipend': 10000,
#     'max_duration': 6,
# }

# user_recommendations = advanced_recommender.recommend_for_user(user_profile, top_n=3)
# print(f"Found {len(user_recommendations)} personalized recommendations")
# for i, rec in enumerate(user_recommendations, 1):
#     print(f"  {i}. {rec['role']} at {rec['company']} (Score: {rec['similarity_score']:.3f})")

# # 4.4 Build API Interface
# print("\n4.4 Building API Interface...")

# class RecommendationAPI:
#     """Lightweight API interface"""
    
#     def __init__(self, recommender):
#         self.recommender = recommender
#         self.endpoints = {
#             'health': self.health_check,
#             'recommend': self.recommend,
#             'internship': self.get_internship,
#             'search': self.search,
#             'stats': self.stats
#         }
    
#     def health_check(self, params=None):
#         return {
#             'status': 'healthy',
#             'timestamp': datetime.now().isoformat(),
#             'internships_count': len(self.recommender.df)
#         }
    
#     def recommend(self, params):
#         try:
#             recommendation_type = params.get('type', 'skills')
#             top_n = int(params.get('top_n', 10))
            
#             if recommendation_type == 'skills':
#                 skills = params.get('skills', [])
#                 if isinstance(skills, str):
#                     skills = [s.strip() for s in skills.split(',')]
#                 results = self.recommender.recommender.recommend_by_skills(skills, top_n)
#             else:
#                 return {'error': 'Invalid recommendation type'}
            
#             return {
#                 'status': 'success',
#                 'count': len(results),
#                 'recommendations': results
#             }
            
#         except Exception as e:
#             return {'error': str(e)}
    
#     def get_internship(self, params):
#         try:
#             internship_id = int(params.get('id', 0))
#             details = self.recommender.get_internship_details(internship_id)
            
#             if not details:
#                 return {'error': 'Internship not found'}
            
#             return {
#                 'status': 'success',
#                 'internship': details
#             }
            
#         except Exception as e:
#             return {'error': str(e)}
    
#     def search(self, params):
#         try:
#             query = params.get('query', '')
#             filters = params.get('filters', {})
#             top_n = int(params.get('top_n', 20))
            
#             results = self.recommender.search_internships(query, filters, top_n)
            
#             return {
#                 'status': 'success',
#                 'count': len(results),
#                 'results': results
#             }
            
#         except Exception as e:
#             return {'error': str(e)}
    
#     def stats(self, params=None):
#         try:
#             df = self.recommender.df
            
#             stats = {
#                 'total_internships': len(df),
#                 'total_companies': df['Company Name'].nunique(),
#                 'average_stipend': float(df['Stipend Amount'].mean()) if 'Stipend Amount' in df else 0,
#                 'average_duration': float(df['Duration'].mean()) if 'Duration' in df else 0
#             }
            
#             return {
#                 'status': 'success',
#                 'stats': stats
#             }
            
#         except Exception as e:
#             return {'error': str(e)}
    
#     def handle_request(self, endpoint, params=None):
#         if endpoint not in self.endpoints:
#             return {'error': 'Endpoint not found'}
        
#         return self.endpoints[endpoint](params)

# # Initialize the API
# api = RecommendationAPI(advanced_recommender)
# print("API interface initialized")

# # 4.5 Test the API interface
# print("\n4.5 Testing API interface...")

# # Test endpoints
# test_cases = [
#     ('health', None),
#     ('recommend', {'type': 'skills', 'skills': 'python,ml', 'top_n': '2'}),
#     ('stats', None)
# ]

# for endpoint, params in test_cases:
#     print(f"\nTesting {endpoint}:")
#     result = api.handle_request(endpoint, params)
#     print(f"  Status: {result.get('status', 'error')}")
#     if 'count' in result:
#         print(f"  Results: {result['count']}")

# # 4.6 Create production-ready files
# print("\n4.6 Creating production-ready files...")

# # Create simple API server file
# app_content = '''import json
# import joblib
# import pandas as pd
# from datetime import datetime

# class RecommendationAPI:
#     def __init__(self, recommender_path='internship_recommender.pkl'):
#         try:
#             recommender_data = joblib.load(recommender_path)
#             self.recommender = recommender_data['recommender']
#             print("Recommender loaded successfully")
#         except Exception as e:
#             print(f"Error loading recommender: {e}")
#             self.recommender = None
    
#     def health_check(self):
#         return {'status': 'healthy', 'timestamp': datetime.now().isoformat()}
    
#     def recommend_skills(self, skills, top_n=5):
#         if self.recommender is None:
#             return {'error': 'Recommender not loaded'}
        
#         try:
#             results = self.recommender.recommend_by_skills(skills, top_n)
#             return {
#                 'status': 'success',
#                 'count': len(results),
#                 'recommendations': results
#             }
#         except Exception as e:
#             return {'error': str(e)}
    
#     def run_demo(self):
#         print("Internship Recommendation System Demo")
#         print("Enter skills separated by commas (or 'quit' to exit)")
        
#         while True:
#             try:
#                 user_input = input("Enter skills: ").strip()
#                 if user_input.lower() == 'quit':
#                     break
                
#                 skills = [s.strip() for s in user_input.split(',')]
#                 result = self.recommend_skills(skills, 3)
                
#                 if result['status'] == 'success':
#                     print(f"Found {result['count']} recommendations:")
#                     for i, rec in enumerate(result['recommendations'], 1):
#                         print(f"{i}. {rec['role']} at {rec['company']} (Score: {rec['similarity_score']:.3f})")
#                 else:
#                     print(f"Error: {result['error']}")
                    
#             except KeyboardInterrupt:
#                 break
#             except Exception as e:
#                 print(f"Error: {e}")

# if __name__ == "__main__":
#     api = RecommendationAPI()
#     if api.recommender:
#         api.run_demo()
# '''

# # Write files with explicit encoding
# try:
#     with open('api_server.py', 'w', encoding='utf-8') as f:
#         f.write(app_content)
#     print("Created api_server.py")
# except Exception as e:
#     print(f"Error creating file: {e}")

# # Create requirements file
# requirements_content = """pandas>=2.0.0
# numpy>=1.24.0
# scikit-learn>=1.3.0
# joblib>=1.3.0
# scipy>=1.11.0
# """

# try:
#     with open('requirements.txt', 'w', encoding='utf-8') as f:
#         f.write(requirements_content)
#     print("Created requirements.txt")
# except Exception as e:
#     print(f"Error creating file: {e}")

# # 4.7 Final summary
# print("\n" + "="*60)
# print("RECOMMENDATION ENGINE COMPLETED SUCCESSFULLY!")
# print("="*60)
# print("Advanced recommendation functions")
# print("User profile-based recommendations")
# print("Search functionality with filters")
# print("Lightweight API interface")
# print("Production-ready files created")

# print("\nTo use the recommendation system:")
# print("1. Run: python api_server.py")
# print("2. Enter skills when prompted")
# print("3. View recommendations")

# print("\nAvailable features:")
# print("- Skill-based recommendations")
# print("- User profile matching")
# print("- Company diversity")
# print("- Search and filtering")
# print("- API interface")

# print("\nRecommendation system is now ready to use!")






# lightweight_api.py
import json
import joblib
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import ast

class LightweightRecommender:
    def __init__(self, data_path='processed_internship_data.csv'):
        """Initialize with minimal memory footprint"""
        print("Loading lightweight recommender...")
        
        # Load only essential columns to save memory
        essential_cols = [
            'Internship Id', 'Role', 'Company Name', 'Skills', 
            'Stipend Amount', 'Location', 'Duration', 'Intern Type'
        ]
        
        self.df = pd.read_csv(data_path, usecols=essential_cols)
        print(f"Loaded {len(self.df)} internships")
        
        # Preprocess skills for efficient matching
        self.df['Skills_Processed'] = self.df['Skills'].apply(self._process_skills)
        
        # Create TF-IDF matrix for skill matching
        self.tfidf = TfidfVectorizer(
            max_features=100,  # Reduced for memory efficiency
            stop_words='english',
            min_df=2,
            max_df=0.8
        )
        
        skills_text = self.df['Skills_Processed'].fillna('')
        self.tfidf_matrix = self.tfidf.fit_transform(skills_text)
        print("TF-IDF matrix created")
    
    def _process_skills(self, skills_str):
        """Convert skills string to processed text"""
        if pd.isna(skills_str) or skills_str == '[]':
            return ''
        
        try:
            if isinstance(skills_str, str) and skills_str.startswith('['):
                skills_list = ast.literal_eval(skills_str)
                if isinstance(skills_list, list):
                    return ' '.join([str(s).lower().strip() for s in skills_list])
            return str(skills_str).lower()
        except:
            return str(skills_str).lower()
    
    def recommend_by_skills(self, skills, top_n=5):
        """Lightweight skill-based recommendation"""
        if not skills:
            return []
        
        # Process input skills
        query_text = ' '.join([str(s).lower().strip() for s in skills])
        query_vector = self.tfidf.transform([query_text])
        
        # Calculate similarities
        similarities = cosine_similarity(query_vector, self.tfidf_matrix).flatten()
        
        # Get top recommendations
        top_indices = similarities.argsort()[-top_n:][::-1]
        
        results = []
        for idx in top_indices:
            internship = self.df.iloc[idx]
            results.append({
                'internship_id': int(internship['Internship Id']),
                'role': str(internship['Role']),
                'company': str(internship['Company Name']),
                'similarity_score': float(similarities[idx]),
                'stipend_amount': float(internship['Stipend Amount']) if not pd.isna(internship['Stipend Amount']) else None,
                'location': str(internship['Location']),
                'duration': int(internship['Duration']) if not pd.isna(internship['Duration']) else None,
                'intern_type': str(internship['Intern Type']) if not pd.isna(internship['Intern Type']) else None
            })
        
        return results
    
    def search_internships(self, query, filters=None, top_n=10):
        """Lightweight search functionality"""
        if filters is None:
            filters = {}
        
        query = query.lower().strip()
        results = []
        
        for _, internship in self.df.iterrows():
            score = 0
            
            # Text matching
            search_fields = ['Role', 'Company Name', 'Location', 'Skills_Processed']
            for field in search_fields:
                field_value = str(internship.get(field, '')).lower()
                if query in field_value:
                    score += 1
            
            # Apply filters
            passes_filters = True
            
            if 'min_stipend' in filters:
                stipend = internship.get('Stipend Amount', 0)
                if pd.isna(stipend) or stipend < filters['min_stipend']:
                    passes_filters = False
            
            if 'location' in filters:
                location = str(internship.get('Location', '')).lower()
                if not any(loc.lower() in location for loc in filters['location']):
                    passes_filters = False
            
            if score > 0 and passes_filters:
                results.append({
                    'internship_id': int(internship['Internship Id']),
                    'role': str(internship['Role']),
                    'company': str(internship['Company Name']),
                    'match_score': score,
                    'stipend_amount': float(internship['Stipend Amount']) if not pd.isna(internship['Stipend Amount']) else None,
                    'location': str(internship['Location'])
                })
        
        results.sort(key=lambda x: x['match_score'], reverse=True)
        return results[:top_n]
    
    def get_internship_details(self, internship_id):
        """Get details for a specific internship"""
        internship = self.df[self.df['Internship Id'] == internship_id]
        if len(internship) == 0:
            return None
        
        internship = internship.iloc[0]
        return {
            'internship_id': int(internship['Internship Id']),
            'role': str(internship['Role']),
            'company': str(internship['Company Name']),
            'location': str(internship['Location']),
            'duration': int(internship['Duration']) if not pd.isna(internship['Duration']) else None,
            'stipend_amount': float(internship['Stipend Amount']) if not pd.isna(internship['Stipend Amount']) else None,
            'intern_type': str(internship['Intern Type']) if not pd.isna(internship['Intern Type']) else None,
            'skills': self._get_skills_list(internship['Skills'])
        }
    
    def _get_skills_list(self, skills_str):
        """Extract skills as list"""
        try:
            if isinstance(skills_str, str) and skills_str.startswith('['):
                return ast.literal_eval(skills_str)
            return [skills_str]
        except:
            return [skills_str]
    
    def get_stats(self):
        """Get basic statistics"""
        return {
            'total_internships': len(self.df),
            'total_companies': self.df['Company Name'].nunique(),
            'average_stipend': float(self.df['Stipend Amount'].mean()) if not self.df['Stipend Amount'].isna().all() else 0,
            'average_duration': float(self.df['Duration'].mean()) if not self.df['Duration'].isna().all() else 0
        }


class LightweightAPI:
    def __init__(self, recommender):
        self.recommender = recommender
        self.endpoints = {
            'health': self.health_check,
            'recommend': self.recommend,
            'details': self.get_details,
            'search': self.search,
            'stats': self.get_stats
        }
    
    def health_check(self):
        return {
            'status': 'healthy',
            'timestamp': datetime.now().isoformat(),
            'internships_count': len(self.recommender.df)
        }
    
    def recommend(self, params):
        try:
            skills = params.get('skills', [])
            if isinstance(skills, str):
                skills = [s.strip() for s in skills.split(',')]
            
            top_n = min(int(params.get('top_n', 5)), 20)  # Limit to 20 for safety
            
            results = self.recommender.recommend_by_skills(skills, top_n)
            
            return {
                'status': 'success',
                'count': len(results),
                'recommendations': results
            }
            
        except Exception as e:
            return {'error': f'Recommendation error: {str(e)}'}
    
    def get_details(self, params):
        try:
            internship_id = int(params.get('id', 0))
            details = self.recommender.get_internship_details(internship_id)
            
            if not details:
                return {'error': 'Internship not found'}
            
            return {
                'status': 'success',
                'internship': details
            }
            
        except Exception as e:
            return {'error': f'Details error: {str(e)}'}
    
    def search(self, params):
        try:
            query = params.get('query', '').strip()
            top_n = min(int(params.get('top_n', 10)), 20)
            
            # Parse filters
            filters = {}
            if 'min_stipend' in params:
                filters['min_stipend'] = float(params['min_stipend'])
            if 'location' in params:
                filters['location'] = [loc.strip() for loc in params['location'].split(',')]
            
            results = self.recommender.search_internships(query, filters, top_n)
            
            return {
                'status': 'success',
                'count': len(results),
                'results': results
            }
            
        except Exception as e:
            return {'error': f'Search error: {str(e)}'}
    
    def get_stats(self):
        try:
            stats = self.recommender.get_stats()
            return {
                'status': 'success',
                'stats': stats
            }
        except Exception as e:
            return {'error': f'Stats error: {str(e)}'}
    
    def handle_request(self, endpoint, params=None):
        if endpoint not in self.endpoints:
            return {'error': 'Endpoint not found'}
        
        if params is None:
            params = {}
        
        return self.endpoints[endpoint](params)


# Flask API Server (minimal version)
try:
    from flask import Flask, request, jsonify
    
    app = Flask(__name__)
    recommender = LightweightRecommender()
    api = LightweightAPI(recommender)
    
    @app.route('/health', methods=['GET'])
    def health():
        return jsonify(api.health_check())
    
    @app.route('/recommend', methods=['GET'])
    def recommend():
        params = request.args.to_dict()
        return jsonify(api.recommend(params))
    
    @app.route('/internship/<int:internship_id>', methods=['GET'])
    def internship_details(internship_id):
        return jsonify(api.get_details({'id': internship_id}))
    
    @app.route('/search', methods=['GET'])
    def search():
        params = request.args.to_dict()
        return jsonify(api.search(params))
    
    @app.route('/stats', methods=['GET'])
    def stats():
        return jsonify(api.get_stats())
    
    print("Flask API routes configured")
    
except ImportError:
    print("Flask not installed - API server mode disabled")


# Command-line interface
def run_cli():
    """Run a simple command-line interface"""
    print("Internship Recommendation System")
    print("=" * 40)
    
    recommender = LightweightRecommender()
    api = LightweightAPI(recommender)
    
    while True:
        print("\nOptions:")
        print("1. Recommend by skills")
        print("2. Search internships")
        print("3. Get internship details")
        print("4. Show statistics")
        print("5. Exit")
        
        choice = input("\nEnter your choice (1-5): ").strip()
        
        if choice == '1':
            skills_input = input("Enter skills (comma-separated): ").strip()
            if skills_input:
                skills = [s.strip() for s in skills_input.split(',')]
                result = api.recommend({'skills': skills, 'top_n': '5'})
                self._display_recommendations(result)
        
        elif choice == '2':
            query = input("Enter search query: ").strip()
            if query:
                result = api.search({'query': query, 'top_n': '10'})
                self._display_search_results(result)
        
        elif choice == '3':
            try:
                internship_id = int(input("Enter internship ID: ").strip())
                result = api.get_details({'id': internship_id})
                self._display_details(result)
            except ValueError:
                print("Please enter a valid number")
        
        elif choice == '4':
            result = api.get_stats()
            self._display_stats(result)
        
        elif choice == '5':
            print("Goodbye!")
            break
        
        else:
            print("Invalid choice. Please try again.")
    
    def _display_recommendations(self, result):
        if result['status'] == 'success':
            print(f"\nFound {result['count']} recommendations:")
            for i, rec in enumerate(result['recommendations'], 1):
                print(f"{i}. {rec['role']} at {rec['company']}")
                print(f"   Score: {rec['similarity_score']:.3f} | Stipend: {rec['stipend_amount'] or 'N/A'}")
                print(f"   Location: {rec['location']} | Duration: {rec['duration'] or 'N/A'} months")
                print()
        else:
            print(f"Error: {result.get('error', 'Unknown error')}")
    
    def _display_search_results(self, result):
        if result['status'] == 'success':
            print(f"\nFound {result['count']} results:")
            for i, rec in enumerate(result['results'], 1):
                print(f"{i}. {rec['role']} at {rec['company']}")
                print(f"   Match score: {rec['match_score']} | Stipend: {rec['stipend_amount'] or 'N/A'}")
                print(f"   Location: {rec['location']}")
                print()
        else:
            print(f"Error: {result.get('error', 'Unknown error')}")
    
    def _display_details(self, result):
        if result['status'] == 'success':
            internship = result['internship']
            print(f"\nInternship Details:")
            print(f"Role: {internship['role']}")
            print(f"Company: {internship['company']}")
            print(f"Location: {internship['location']}")
            print(f"Duration: {internship['duration'] or 'N/A'} months")
            print(f"Stipend: {internship['stipend_amount'] or 'N/A'}")
            print(f"Type: {internship['intern_type'] or 'N/A'}")
            print(f"Skills: {', '.join(internship['skills'][:5])}{'...' if len(internship['skills']) > 5 else ''}")
        else:
            print(f"Error: {result.get('error', 'Internship not found')}")
    
    def _display_stats(self, result):
        if result['status'] == 'success':
            stats = result['stats']
            print(f"\nSystem Statistics:")
            print(f"Total internships: {stats['total_internships']:,}")
            print(f"Total companies: {stats['total_companies']:,}")
            print(f"Average stipend: ₹{stats['average_stipend']:,.0f}")
            print(f"Average duration: {stats['average_duration']:.1f} months")
        else:
            print(f"Error: {result.get('error', 'Unknown error')}")


if __name__ == "__main__":
    # Create requirements file
    requirements = """pandas>=1.3.0
numpy>=1.21.0
scikit-learn>=1.0.0
joblib>=1.0.0
flask>=2.0.0  # Optional for web API
"""
    
    with open('requirements_lightweight.txt', 'w') as f:
        f.write(requirements)
    print("Created requirements_lightweight.txt")
    
    # Run CLI interface
    run_cli()

Loading lightweight recommender...
Loaded 6642 internships
TF-IDF matrix created
Flask API routes configured
Created requirements_lightweight.txt
Internship Recommendation System
Loading lightweight recommender...
Loaded 6642 internships
TF-IDF matrix created

Options:
1. Recommend by skills
2. Search internships
3. Get internship details
4. Show statistics
5. Exit



Enter your choice (1-5):  python


Invalid choice. Please try again.

Options:
1. Recommend by skills
2. Search internships
3. Get internship details
4. Show statistics
5. Exit



Enter your choice (1-5):  buisness


Invalid choice. Please try again.

Options:
1. Recommend by skills
2. Search internships
3. Get internship details
4. Show statistics
5. Exit



Enter your choice (1-5):  1
Enter skills (comma-separated):  python


NameError: name 'self' is not defined

In [ ]:
# Continue from the previous code...

print("\n=== STEP 5: Recommendation Engine Evaluation ===\n")

# 5.1 Create evaluation framework
print("5.1 Setting up evaluation framework...")

class RecommendationEvaluator:
    def __init__(self, recommender):
        self.recommender = recommender
        self.df = recommender.df
        
    def create_test_cases(self, num_cases=10):
        """Create test cases for evaluation"""
        print(f"Creating {num_cases} test cases...")
        
        test_cases = []
        
        # Get diverse sample of internships for testing
        sample_indices = self.df.sample(min(num_cases, len(self.df))).index
        
        for idx in sample_indices:
            internship = self.df.iloc[idx]
            skills = internship['Skills']
            
            if isinstance(skills, list) and len(skills) > 0:
                # Use actual skills from the internship
                test_skills = skills[:3]  # Use first 3 skills
                test_cases.append({
                    'test_id': len(test_cases) + 1,
                    'input_skills': test_skills,
                    'expected_internship_id': internship['Internship Id'],
                    'expected_role': internship['Role'],
                    'expected_company': internship['Company Name']
                })
        
        return test_cases
    
    def precision_at_k(self, recommendations, expected_id, k=5):
        """Calculate precision@k - whether expected item is in top k recommendations"""
        if not recommendations:
            return 0.0
        
        top_k = recommendations[:k]
        relevant = any(rec['internship_id'] == expected_id for rec in top_k)
        return 1.0 if relevant else 0.0
    
    def mean_average_precision(self, test_cases, k=5):
        """Calculate Mean Average Precision@k"""
        print(f"Calculating MAP@{k}...")
        
        ap_scores = []
        
        for case in test_cases:
            recommendations = self.recommender.recommender.recommend_by_skills(
                case['input_skills'], top_n=k*2
            )
            
            precision_scores = []
            for i in range(1, k+1):
                prec = self.precision_at_k(recommendations, case['expected_internship_id'], i)
                precision_scores.append(prec)
            
            # Average Precision for this test case
            if any(precision_scores):  # If at least one relevant item found
                ap = sum(precision_scores) / k
            else:
                ap = 0.0
                
            ap_scores.append(ap)
        
        map_score = sum(ap_scores) / len(ap_scores) if ap_scores else 0.0
        return map_score
    
    def coverage_metric(self, test_cases, k=10):
        """Calculate what percentage of internships get recommended"""
        print("Calculating coverage metric...")
        
        all_recommended_ids = set()
        
        for case in test_cases:
            recommendations = self.recommender.recommender.recommend_by_skills(
                case['input_skills'], top_n=k
            )
            for rec in recommendations:
                all_recommended_ids.add(rec['internship_id'])
        
        total_internships = len(self.df)
        coverage = len(all_recommended_ids) / total_internships if total_internships > 0 else 0.0
        return coverage
    
    def diversity_metric(self, test_cases, k=5):
        """Calculate diversity of recommendations across companies"""
        print("Calculating diversity metric...")
        
        all_companies = set()
        recommended_companies = set()
        
        for case in test_cases:
            recommendations = self.recommender.recommender.recommend_by_skills(
                case['input_skills'], top_n=k
            )
            for rec in recommendations:
                recommended_companies.add(rec['company'])
        
        # Company diversity: percentage of unique companies recommended
        total_companies = self.df['Company Name'].nunique()
        company_diversity = len(recommended_companies) / total_companies if total_companies > 0 else 0.0
        
        return company_diversity
    
    def generate_evaluation_report(self, test_cases, k_values=[3, 5, 10]):
        """Generate comprehensive evaluation report"""
        print("Generating evaluation report...")
        
        report = {
            'timestamp': datetime.now().isoformat(),
            'total_test_cases': len(test_cases),
            'total_internships': len(self.df),
            'total_companies': self.df['Company Name'].nunique(),
            'metrics': {}
        }
        
        for k in k_values:
            map_score = self.mean_average_precision(test_cases, k)
            coverage = self.coverage_metric(test_cases, k)
            diversity = self.diversity_metric(test_cases, k)
            
            report['metrics'][f'k={k}'] = {
                'map_score': round(map_score, 4),
                'coverage': round(coverage, 4),
                'diversity': round(diversity, 4)
            }
        
        return report
    
    def human_evaluation_template(self, test_cases, num_cases=5):
        """Generate template for human evaluation"""
        print("Generating human evaluation template...")
        
        evaluation_template = {
            'evaluation_id': f"eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            'instructions': "Rate each recommendation set from 1-5 (1=Poor, 5=Excellent)",
            'test_cases': []
        }
        
        for i, case in enumerate(test_cases[:num_cases]):
            recommendations = self.recommender.recommender.recommend_by_skills(
                case['input_skills'], top_n=5
            )
            
            evaluation_case = {
                'case_id': i + 1,
                'input_skills': case['input_skills'],
                'expected_internship': {
                    'id': case['expected_internship_id'],
                    'role': case['expected_role'],
                    'company': case['expected_company']
                },
                'recommendations': [
                    {
                        'rank': j + 1,
                        'internship_id': rec['internship_id'],
                        'role': rec['role'],
                        'company': rec['company'],
                        'score': rec.get('similarity_score', 0),
                        'human_rating': None,
                        'comments': None
                    }
                    for j, rec in enumerate(recommendations)
                ]
            }
            
            evaluation_template['test_cases'].append(evaluation_case)
        
        return evaluation_template

# 5.2 Initialize evaluator and run evaluation
print("5.2 Running evaluation...")

evaluator = RecommendationEvaluator(advanced_recommender)

# Create test cases
test_cases = evaluator.create_test_cases(num_cases=20)
print(f"Created {len(test_cases)} test cases")

# Run automated evaluation
evaluation_report = evaluator.generate_evaluation_report(test_cases, k_values=[3, 5, 8])

print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)

for k, metrics in evaluation_report['metrics'].items():
    print(f"\nMetrics for {k}:")
    print(f"  MAP Score:    {metrics['map_score']:.3f}")
    print(f"  Coverage:     {metrics['coverage']:.3f}")
    print(f"  Diversity:    {metrics['diversity']:.3f}")

# 5.3 Generate human evaluation template
print("\n5.3 Generating human evaluation materials...")

human_eval = evaluator.human_evaluation_template(test_cases, num_cases=3)

# Save human evaluation template
try:
    with open('human_evaluation_template.json', 'w', encoding='utf-8') as f:
        json.dump(human_eval, f, indent=2, ensure_ascii=False)
    print("Saved human_evaluation_template.json")
except Exception as e:
    print(f"Error saving human evaluation template: {e}")

# 5.4 Create A/B testing framework
print("\n5.4 Creating A/B testing framework...")

class ABTestingFramework:
    def __init__(self, recommender):
        self.recommender = recommender
        self.experiments = {}
    
    def create_experiment(self, experiment_id, variants):
        """Create an A/B test experiment"""
        self.experiments[experiment_id] = {
            'variants': variants,
            'results': {},
            'start_time': datetime.now().isoformat()
        }
        print(f"Created experiment: {experiment_id}")
    
    def recommend_variant(self, experiment_id, user_id, skills, variant_name):
        """Get recommendations for a specific variant"""
        if experiment_id not in self.experiments:
            return None
        
        variant = self.experiments[experiment_id]['variants'].get(variant_name)
        if not variant:
            return None
        
        # Apply variant-specific parameters
        top_n = variant.get('top_n', 10)
        diversity = variant.get('diversity_factor', 0.3)
        
        if variant_name == 'baseline':
            return self.recommender.recommender.recommend_by_skills(skills, top_n)
        elif variant_name == 'diverse':
            return self.recommender.recommend_with_diversity(skills, top_n, diversity)
        elif variant_name == 'hybrid':
            return self.recommender.recommend_hybrid(
                skills=skills,
                min_stipend=variant.get('min_stipend', 0),
                max_duration=variant.get('max_duration', 12),
                top_n=top_n
            )
        
        return None
    
    def track_conversion(self, experiment_id, user_id, variant_name, internship_id, action='click'):
        """Track user actions for A/B testing"""
        if experiment_id not in self.experiments:
            return False
        
        if 'results' not in self.experiments[experiment_id]:
            self.experiments[experiment_id]['results'] = {}
        
        if variant_name not in self.experiments[experiment_id]['results']:
            self.experiments[experiment_id]['results'][variant_name] = {
                'clicks': 0,
                'applications': 0,
                'users': set(),
                'conversions': 0
            }
        
        variant_results = self.experiments[experiment_id]['results'][variant_name]
        variant_results['users'].add(user_id)
        
        if action == 'click':
            variant_results['clicks'] += 1
        elif action == 'application':
            variant_results['applications'] += 1
            variant_results['conversions'] += 1
        
        return True
    
    def calculate_metrics(self, experiment_id):
        """Calculate A/B test metrics"""
        if experiment_id not in self.experiments:
            return None
        
        results = {}
        experiment = self.experiments[experiment_id]
        
        for variant_name, variant_data in experiment['results'].items():
            users = len(variant_data['users'])
            clicks = variant_data['clicks']
            applications = variant_data['applications']
            
            click_through_rate = clicks / users if users > 0 else 0
            conversion_rate = applications / users if users > 0 else 0
            
            results[variant_name] = {
                'users': users,
                'clicks': clicks,
                'applications': applications,
                'click_through_rate': round(click_through_rate, 4),
                'conversion_rate': round(conversion_rate, 4)
            }
        
        return results

# Initialize A/B testing framework
ab_testing = ABTestingFramework(advanced_recommender)

# Create sample A/B test experiment
experiment_variants = {
    'baseline': {
        'description': 'Basic skill-based recommendations',
        'top_n': 5
    },
    'diverse': {
        'description': 'Diversity-enhanced recommendations',
        'top_n': 5,
        'diversity_factor': 0.4
    },
    'hybrid': {
        'description': 'Hybrid recommendations with filters',
        'top_n': 5,
        'min_stipend': 5000,
        'max_duration': 6
    }
}

ab_testing.create_experiment('recommendation_style', experiment_variants)
print("A/B testing framework initialized")

# 5.5 Save evaluation results
print("\n5.5 Saving evaluation results...")

# Save automated evaluation report
try:
    with open('evaluation_report.json', 'w', encoding='utf-8') as f:
        json.dump(evaluation_report, f, indent=2, ensure_ascii=False)
    print("Saved evaluation_report.json")
except Exception as e:
    print(f"Error saving evaluation report: {e}")

# Save A/B test configuration
ab_config = {
    'experiments': ab_testing.experiments,
    'framework_version': '1.0',
    'created_at': datetime.now().isoformat()
}

try:
    with open('ab_testing_config.json', 'w', encoding='utf-8') as f:
        json.dump(ab_config, f, indent=2, ensure_ascii=False)
    print("Saved ab_testing_config.json")
except Exception as e:
    print(f"Error saving A/B test config: {e}")

# 5.6 Final evaluation summary
print("\n" + "="*60)
print("EVALUATION COMPLETED SUCCESSFULLY!")
print("="*60)

print("\nEvaluation Metrics Summary:")
for k, metrics in evaluation_report['metrics'].items():
    print(f"{k}:")
    print(f"  MAP: {metrics['map_score']:.3f} | Coverage: {metrics['coverage']:.3f} | Diversity: {metrics['diversity']:.3f}")

print("\nFiles Created:")
print("✓ evaluation_report.json - Automated metrics")
print("✓ human_evaluation_template.json - For manual rating")
print("✓ ab_testing_config.json - A/B testing framework")

print("\nNext Steps:")
print("1. Review evaluation metrics")
print("2. Conduct human evaluation using the template")
print("3. Set up A/B testing in production")
print("4. Monitor and iterate based on results")

print("\nThe recommendation engine is now fully evaluated and ready for production deployment!")


=== STEP 5: Recommendation Engine Evaluation ===

5.1 Setting up evaluation framework...
5.2 Running evaluation...
Creating 20 test cases...
Created 20 test cases
Generating evaluation report...
Calculating MAP@3...
  - Finding recommendations for skills: ['Copywriting', 'Creative Writing', 'Proofreading']
  - Finding recommendations for skills: ['Adobe After Effects', 'Adobe Creative Suite', 'Adobe Illustrator']
  - Finding recommendations for skills: ['English Proficiency (Spoken)', 'English Proficiency (Written)']
  - Finding recommendations for skills: ['English Proficiency (Spoken)', 'Marketing', 'MS-Excel']
  - Finding recommendations for skills: ['Certificate', 'Flexible work hours', 'Informal dress code']
  - Finding recommendations for skills: ['Effective Communication', 'MS-Excel']
  - Finding recommendations for skills: ['Effective Communication', 'English Proficiency (Spoken)']
  - Finding recommendations for skills: ['Adobe Illustrator', 'Adobe Indesign', 'Adobe Photoshop